In [1]:
!pip install afinn
!python -m textblob.download_corpora
!pip install -U textblob
!pip install vaderSentiment
!pip install pyemd
!pip install unidecode

     |████████████████████████████████| 52 kB 1.1 MB/s 
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53447 sha256=fc69d54f74d1ad256b71e4570b2610d0ecda3da929cec1bc22b57fdc9d4c07de
  Stored in directory: /root/.cache/pip/wheels/9d/16/3a/9f0953027434eab5dadf3f33ab3298fa95afa8292fcf7aba75
Successfully built afinn
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data

In [2]:
import re
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
from unidecode import unidecode
import pandas as pd
import bz2
import gensim
import warnings
import numpy as np
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

warnings.filterwarnings('ignore')
tqdm_notebook.pandas()

import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  app.launch_new_instance()


# Carregando os embeddings

Aqui vamos utilizar os embeddings para realizar as seguintes atividades:

- análise de simlaridade
- classificação de documentos

<b> Carregue os embeddings treinados, como vimos na Aula 2. É o mesmo arquivo que iremos utilizar</b>

Link: https://drive.google.com/open?id=1zI8pGfbUHuU_0wY_FV4tD6w6ZCUJTQbh

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = "/content/drive/MyDrive/Colab Notebooks/Bases/ptwiki_20180420_100d.txt"
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(path, binary=False)

# Similaridade de Documentos

Para realizar a similaridade entre documentos, utilize as frases abaixo:

In [5]:
frase1 = "Excelente produto chegou antes do prazo indico e recomendo produto bom pois já testei e foi mais que aprovado" 
frase2 = "SUPER RECOMENDO, PREÇO, QUALIDADE #BRASTEMP, EFICIÊNCIA NA ENTREGA, E FACILIDADE DE PAGAMENTO. MUITO BOM!!!"
frase3 = "A tampa do fogão veio com problemas com o pino de encaixe solto e precisa de reparos"
frase4 = "Fogão ótimo!"

## Distância de Jaccard

<b> Atividade </b>

1) Faça um método que calcule a similaridade de Jaccard e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: lembrando que você precisa aplicar um pre-processamento nessas frases antes de aplicar o método.
Faça:

- Lower
- Remoção StopWords
- Remoção Pontuação
- Tokenização

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
def pre_processamento(corpus):
  #tokenização
  corpus_new = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
  #lowcase
  corpus_new = [t.lower() for t in corpus_new]
  #remover stopwords
  corpus_pt_stopworld = stopwords.words('portuguese')
  corpus_new = [t for t in corpus_new if t not in corpus_pt_stopworld]
  #remover pontuações
  corpus_new = [t for t in corpus_new if t not in string.punctuation]
  return corpus_new

In [8]:
frase1_limpa = pre_processamento(frase1)
frase2_limpa = pre_processamento(frase2)
frase3_limpa = pre_processamento(frase3)
frase4_limpa = pre_processamento(frase4)

In [9]:
def jacard(f1, f2):
  return len(set(f1).intersection(set(f2))) / len(set(f1).union(set(f2)))

In [10]:
jacard(frase1_limpa, frase2_limpa)

0.10526315789473684

In [11]:
jacard(frase1_limpa, frase3_limpa)

0.0

In [12]:
jacard(frase2_limpa, frase3_limpa)

0.0

In [13]:
jacard(frase1_limpa, frase4_limpa)

0.0

<b> Atividade </b>

2) Qual par de frase teve maior simlaridade? E qual teve menor? Este resultado faz sentido? Explique.

Baseado na medida de similidade de jaccard temos apenas a comparação entre o 1 e 2 com simiralidade, de restante nenhum outro apresenta simiralidade (baseado na medida de similidade de jaccard), faz sentido pelo fato do modelo não entender que se trata do mesmo assunto, por ser um modelo um tanto "limitado" nesse aspecto.

## Distância de Cosseno

Aqui iremos calcular a distância do cosseno utilizando duas formas, que aprendemos na aula passada, para representar o texto.

- Bag of Words (BOW) 
- Embedding

Observação:

Existem duas formas de trabalhar com o cosseno:

<b> Distância </b>: quanto menor mais perto estão as frases.
<b> Similaridade </b>: quanto maior mais perto estão as frases.

### BOW - Distância do cosseno

<b> Atividade </b>

3) Calcule a distância do cosseno utilizando a representação CountVectorizer e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: no CountVectorizer utilizem as frases já pre-processadas da atividade anterior. Mas para aplicá-las no fit_transform, cada frase deve ser um string (sem estar tokenizada) dentro de uma lista.

```python
#exemplo
distance.cosine(frase1, frase2)
```

In [14]:
vect_bag = CountVectorizer()

In [15]:
doc_bag = vect_bag.fit_transform([' '.join(frase1_limpa), \
                                  ' '.join(frase2_limpa), \
                                  ' '.join(frase3_limpa), \
                                  ' '.join(frase4_limpa)])

In [16]:
vect_bag.get_feature_names()

['antes',
 'aprovado',
 'bom',
 'brastemp',
 'chegou',
 'eficiência',
 'encaixe',
 'entrega',
 'excelente',
 'facilidade',
 'fogão',
 'indico',
 'pagamento',
 'pino',
 'pois',
 'prazo',
 'precisa',
 'preço',
 'problemas',
 'produto',
 'qualidade',
 'recomendo',
 'reparos',
 'solto',
 'super',
 'tampa',
 'testei',
 'veio',
 'ótimo']

In [17]:
frase1_bow = doc_bag.todense()[0]
frase2_bow = doc_bag.todense()[1]
frase3_bow = doc_bag.todense()[2]
frase4_bow = doc_bag.todense()[3]

In [18]:
distance.cosine(frase1_bow, frase2_bow)

0.8309691490542968

In [19]:
distance.cosine(frase1_bow, frase3_bow)

1.0

In [20]:
distance.cosine(frase2_bow, frase3_bow)

1.0

In [21]:
distance.cosine(frase1_bow, frase4_bow)

1.0

### Embedding - Distância do cosseno

Para calcular o embedding de cada uma das frases, utilize o modelo carregado inicialmente. 

Cada palavra tem um vetor, para formar o embedding da frase tire a média de todos os vetores.

Utilize as frases já pre-processadas

<b> Atividade </b> 

4) Calcule a distância do cosseno utilizando a representação Embedding e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4


In [22]:
def calcula_embedding(frase):
  return np.mean(np.array([word_vectors[palavra] for palavra in frase]), axis=0)

In [23]:
frase1_emb = calcula_embedding(frase1_limpa)
frase2_emb = calcula_embedding(frase2_limpa)
frase3_emb = calcula_embedding(frase3_limpa)
frase4_emb = calcula_embedding(frase4_limpa)

In [24]:
distance.cosine(frase1_emb, frase2_emb)

0.16096973419189453

In [25]:
distance.cosine(frase1_emb, frase3_emb)

0.2234203815460205

In [26]:
distance.cosine(frase2_emb, frase3_emb)

0.2526938319206238

In [27]:
distance.cosine(frase1_emb, frase4_emb)

0.29043054580688477

<b>Atividade </b>

5) Qual diferença entre as representações do exercício 3 e 4? Qual faz mais sentido? Explique.

No exercicio 4 usamos um embedding treinado já e no exercicio 3 não, então podemos claramente ver que no exercicio 4 temos respostas mais concretas.

## WMD

O WMD já está incorporado ao Word2Vec

<b> Atividade </b>

6) Calcule a distância WMD e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: use a variável já tokenizada.

In [29]:
word_vectors.wmdistance(frase1_limpa, frase2_limpa)

3.0725697404938526

In [30]:
word_vectors.wmdistance(frase1_limpa, frase3_limpa)

3.759219170455064

In [31]:
word_vectors.wmdistance(frase2_limpa, frase3_limpa)

3.9048993635316385

In [32]:
word_vectors.wmdistance(frase1_limpa, frase4_limpa)

3.851623338336251

<b> Atividade </b>

7) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.

O par que teve a maior similaridade foi a 1 com a 2, que até agora se mostrou de todos os modelos igual, já a menor se trata da 2 e 3.

# Classificação de Documentos

A clssificação de documentos é muito útil em vários aspectos. Um dos tipos de classificação de texto é a análise de sentimentos.

A fim de ilustrar a classificação de documentos iremos criar um modelo para classificar uma frase como positiva ou negativa.

## Dataset

<b> Atividade </b>

8) Carregue o dataset com o pandas e depois dê o head no dataframe.


Link download: https://drive.google.com/open?id=15azJWdEEPGsXQGiDmEOseTBJcquWvBQc

<b> Este dataset é sobre revisões de filmes do IMDB. </b>

In [33]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Bases/imdb-reviews-pt-br.csv")

In [34]:
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


## Representação dos dados

O sentimento positivo e negativo iremos binarizar cada um deles. Seja 1 positivo e 0 negativo.

Iremos representar o texto de duas formas:

- Bag of Words (BOW)
- Embedding

Depois iremos comparar o resultado de cada um deles.

### Representação Target

<b> Atividade </b>

9) Faça a representação dos sentimentos. 1 positivo; 0 negativo

In [36]:
target = df['sentiment'].replace(['neg', 'pos'], [0 , 1])

In [ ]:
target

### Bag of Words (BOW)

<b> Atividade </b>

10) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords``` no dataframe para armazenar este dado processado):

- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto sem tokenização, ou seja uma string

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [40]:
def pre_processamento_texto(corpus):
  #tokenização
  corpus_new = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
  #remover stopwords
  corpus_pt_stopworld = stopwords.words('portuguese')
  corpus_new = [t for t in corpus_new if t not in corpus_pt_stopworld]
  #remover pontuações
  corpus_new = [t for t in corpus_new if t not in string.punctuation]
  #remover números
  corpus_new = [re.sub(r'\d', '', t) for t in corpus_new]

  corpus_new_str = ' '.join(corpus_new).lower()
  return corpus_new_str

In [41]:
df['text_pt_limpo'] = df['text_pt'].progress_apply(pre_processamento_texto)

  0%|          | 0/49459 [00:00<?, ?it/s]

In [42]:
df['text_pt_limpo']

0        mais vez sr costner arrumou filme tempo necess...
1        este exemplo motivo maioria filmes ação mesmos...
2        primeiro tudo odeio raps imbecis poderiam agir...
3        nem beatles puderam escrever músicas todos gos...
4        filmes fotos latão palavra apropriada verdade ...
                               ...                        
49454    como média votos baixa fato funcionário locado...
49455    o enredo algumas reviravoltas infelizes inacre...
49456    estou espantado forma filme maioria outros méd...
49457    a christmas together realmente veio antes temp...
49458    o drama romântico classe trabalhadora diretor ...
Name: text_pt_limpo, Length: 49459, dtype: object

<b> Atividade </b>

11) Aplique a representação do texto processado anteriormente com CountVectorizer.

In [43]:
vect_bag = CountVectorizer()

In [44]:
x_bag = vect_bag.fit_transform(df['text_pt_limpo'])

In [45]:
x_bag

<49459x127624 sparse matrix of type '<class 'numpy.int64'>'
	with 5267081 stored elements in Compressed Sparse Row format>

### Embedding

<b> Atividade </b>

12) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords_token``` no dataframe para armazenar este dado processado):

- Aplique lower
- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto com tokenização

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [46]:
def pre_processamento12(corpus):
  #3tokenização
  corpus_new = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
  #lowcase
  corpus_new = [t.lower() for t in corpus_new]
  #remover stopwords
  corpus_pt_stopworld = stopwords.words('portuguese')
  corpus_new = [t for t in corpus_new if t not in corpus_pt_stopworld]
  #remover números
  corpus_new = [re.sub(r'\d', '', t) for t in corpus_new]
  #remover pontuações
  corpus_new = [t for t in corpus_new if t not in string.punctuation]

  return corpus_new

In [49]:
df['text_pt_sem_stopwords_token'] = df['text_pt'].progress_apply(lambda x: pre_processamento12(x))

  0%|          | 0/49459 [00:00<?, ?it/s]


<b> Atividade </b>

13) Aplique a representação do texto com Embeddings. Cada palavra tem um embedding, o embedding da frase é a média de todos embeddings.

In [50]:
def calcula_embedding13(tokens):
  return np.mean([word_vectors[t] for t in tokens if t in word_vectors.vocab], axis=0)

In [52]:
x_embedding = df['text_pt_sem_stopwords_token'].progress_apply(lambda x: calcula_embedding13(x))

  0%|          | 0/49459 [00:00<?, ?it/s]

In [53]:
x_embedding

0        [0.14301138, 0.22025318, -0.047075942, -0.2170...
1        [0.16303213, 0.22678684, -0.05451168, -0.20547...
2        [0.21659917, 0.14430527, -0.0034280657, -0.197...
3        [0.2596114, 0.13605723, -0.07343373, -0.208256...
4        [0.22052252, 0.087914094, -0.06986406, -0.1659...
                               ...                        
49454    [0.16023268, 0.1474827, -0.08474423, -0.103290...
49455    [0.17767419, 0.1423968, -0.081170976, -0.22615...
49456    [0.18305564, 0.18313898, -0.062747374, -0.1887...
49457    [0.18229564, 0.15024172, -0.08812309, -0.22163...
49458    [0.26787597, 0.16923124, -0.06144687, -0.18159...
Name: text_pt_sem_stopwords_token, Length: 49459, dtype: object

## Treinamento

### CountVectorizer


<b> Atividade </b>

14) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

```python
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bag, target,random_state=123)
```

In [59]:
x_train_bow, x_test_bow, y_train_bow, y_test_bow = train_test_split(x_bag, target, random_state = 123)


<b> Atividade </b>

15) Treine com uma regressão logística

In [60]:
modelo_bow = LogisticRegression()
modelo_bow.fit(x_train_bow, y_train_bow)

LogisticRegression()


<b> Atividade </b>

16) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

In [61]:
y_pred = modelo_bow.predict(x_test_bow)

In [62]:
print(classification_report(y_test_bow, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.87      0.88      6112
           1       0.87      0.89      0.88      6253

    accuracy                           0.88     12365
   macro avg       0.88      0.88      0.88     12365
weighted avg       0.88      0.88      0.88     12365



### Embedding


<b> Atividade </b>

17) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

Verifique o shape do X treino e X teste. Caso eles estejam com apenas uma dimensão, você precisa tranformá-los para duas dimensões, caso contrário ocorrerá erro no treinamento.

In [82]:
x_train_embedding, x_test_embedding, y_train_embedding, y_test_embedding = train_test_split(x_embedding, target, random_state = 123)

In [83]:
x_train_embedding = pd.DataFrame([x for x in x_train_embedding])
x_train_embedding.shape

(37094, 100)

In [84]:
x_test_embedding = pd.DataFrame([x for x in x_test_embedding])
x_test_embedding.shape

(12365, 100)


<b> Atividade </b>

18) Treine com uma regressão logística

In [85]:
modelo_embedding = LogisticRegression()
modelo_embedding.fit(x_train_embedding, y_train_embedding)

LogisticRegression()


<b> Atividade </b>

19) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

#### Calcule as métricas

In [86]:
y_pred = modelo_embedding.predict(x_test_embedding)

In [87]:
print(classification_report(y_test_embedding, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.77      0.77      6112
           1       0.78      0.77      0.77      6253

    accuracy                           0.77     12365
   macro avg       0.77      0.77      0.77     12365
weighted avg       0.77      0.77      0.77     12365




<b> Atividade </b>

20) Compare os resultados obtidos com o BagOfWords e com o Embedding. Explique os possíveis motivos desta diferença.

Nesse caso o BagofWords (88%) foi melhor ao que o Embedding (77%) pois no embedding usamos somente 100 dimensões, e já o BagofWords já usou aproximadamente 127000 dimensões, e além disso pra esse determinado problema o BagofWords trabalha melhor

# Análise de sentimentos

O modelo que criamos anteriormente é para ilustrar como podemos realizar classificação de documentos.
Quando a tarefa é sobre análise de sentimentos, temos duas opções: treinar nosso próprio modelo, como feito anteriormente ou utilizar uma das inúmeras ferramentas prontas.

Vamos testar as seguintes ferramentas:

- Vader
- Textblob
- Affin

Nesta atividade iremos utilizar as duas variáveis abaixo:

In [88]:
texto_neg = df.loc[0, "text_en"]
texto_pos = df.loc[49431, "text_en"]

## Vader

<b> Apenas Inglês </b>

O VADER (Valence Aware Dictionary e sEntiment Reasoner) é uma ferramenta de análise de sentimentos baseada em regras e léxico, especificamente identifica os sentimentos expressos nas mídias sociais.

- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) e (compound score < 0.05)
- negative sentiment: compound score <= -0.05

Mais informações: https://github.com/cjhutto/vaderSentiment


<b> Atividade </b>

21) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(texto)
```

In [89]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [103]:
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(texto_pos)

{'compound': 0.9969, 'neg': 0.084, 'neu': 0.737, 'pos': 0.179}

In [102]:
analyzer.polarity_scores(texto_neg)

{'compound': 0.3958, 'neg': 0.126, 'neu': 0.76, 'pos': 0.114}

## TextBlob

<b> Apenas inglês </b>

https://www.presentslide.in/2019/08/sentiment-analysis-textblob-library.html

<b> Atividade </b>
 
22) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
sentence=TextBlob(texto)
sentence.sentiment
```

In [92]:
from textblob import TextBlob

In [93]:
sentence=TextBlob(texto_pos)
sentence.sentiment

Sentiment(polarity=0.190819118692253, subjectivity=0.6026226012793177)

In [94]:
sentence=TextBlob(texto_neg)
sentence.sentiment

Sentiment(polarity=0.06385964912280702, subjectivity=0.5629824561403508)

## Afinn

- Valor maior que 0 indica sentimento positivo
- Valor menor que 0 indica sentimento negativo

<b> Atividade </b>

23) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
afinn = Afinn()
afinn.score(texto)
```

In [95]:
from afinn import Afinn

In [99]:
afinn = Afinn()
afinn.score(texto_pos)

55.0

In [100]:
afinn.score(texto_neg)

8.0

<b> Atividade </b>

24) Para você, qual ferramenta teve melhor comportamento?  Justifique com exemplos.

Vendo por cima podemos ver que o metodo que mais se aproximou de prever a similaridade entre o positivo e negativo foi o Vader, os outros passaram bem longe comparado a ele.

# Dica:
## Quando for trabalhar com um dataset em inglês, a biblioteca Spacy facilita!

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
import spacy
import pandas as pd

O scpay forne um pacote que já tem série de modelos já treinados em NLP. Inclusive para os embeddings em inglês.

Para mais informações vá em:

https://spacy.io/models/en#en_core_web_md



Com o método abaixo carregamos um dos modelos do spacy:

```python
nlp = spacy.load('en_core_web_md')
```

Para aplicar o modelo, basta passar o texto para o modelo carregado anteriormente:

```python
doc = nlp("This is some text that I am processing with Spacy")
```

Carregue o modelo e imprima doc

In [ ]:
nlp = spacy.load('en_core_web_md')

In [ ]:
doc = nlp("This is some text that I am processing with Spacy")

In [ ]:
doc

Ao aplicar o modelo carregado a variável <b> doc </b> já possui os embeddings de cada uma das palavras e o embedding da frase, que é a média de todos vetores de todas palavras.

```python
#vetor da primeira palavra
doc[0].vector
#vetor agregado pela média - embedding do documento
doc.vector
```
O código abaixo mostra que a média de uma posição em específico dos embeddings de todas as palavras e a posição do embedding do documento possuem o mesmo valor.

In [ ]:
def calcula_media_posicao(x):
    soma = 0
    vector = []
    for i in range(0,len(doc)):
        vector.append(doc[i].vector)    
    
    for v in vector:
        soma += v[x]
    return soma/len(doc)

In [ ]:
round(calcula_media_posicao(10),6)

In [ ]:
round(doc.vector[10], 6)